# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,carnarvon,-24.8667,113.6333,20.45,76,35,6.71,AU,1724434218
1,1,labrador city,52.9463,-66.9114,20.95,46,75,0.00,CA,1724434406
2,2,blackmans bay,-43.0167,147.3167,9.07,92,100,0.45,AU,1724434407
3,3,hawaiian paradise park,19.5933,-154.9731,24.29,98,100,0.45,US,1724434408
4,4,adamstown,-25.0660,-130.1015,18.61,62,100,5.06,PN,1724434409


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values

# Ask user to input parameters for ideal weather conditions
low_temp = float(input("What is your ideal Low Temp?")) #21
high_temp = float(input("What is your ideal High Temp?")) #27
wind_speed = float(input("What is your ideal Max Wind Speed?")) #4.5
cloudiness = float(input("What is your ideal Max Cloudiness?")) # 0 

ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= high_temp) & (city_data_df["Max Temp"] >= low_temp) & 
                                (city_data_df["Wind Speed"] <= wind_speed) &
                                (city_data_df["Cloudiness"] <= cloudiness)] 
                


# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna(how="any")

# Display sample data
ideal_cities_df


What is your ideal Low Temp? 21
What is your ideal High Temp? 27
What is your ideal Max Wind Speed? 10
What is your ideal Max Cloudiness? 33


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,afaahiti,-17.7500,-149.2833,23.78,72,25,7.94,PF,1724434428
48,48,missoula,47.0166,-113.8009,22.26,35,0,2.68,US,1724433648
61,61,bethel,41.3712,-73.4140,26.70,44,0,2.57,US,1724434323
75,75,jamestown,42.0970,-79.2353,24.45,49,0,3.60,US,1724434488
78,78,alice springs,-23.7000,133.8833,22.75,33,1,3.60,AU,1724434491
...,...,...,...,...,...,...,...,...,...,...
515,515,guerrero negro,27.9769,-114.0611,24.66,58,6,6.00,MX,1724434999
518,518,pemba,-12.9740,40.5178,25.39,88,20,6.17,MZ,1724435002
523,523,ilha de mocambique,-15.0342,40.7358,22.70,65,10,3.99,MZ,1724435008
540,540,santanyi,39.3546,3.1291,26.56,69,3,3.85,ES,1724435031


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
21,afaahiti,PF,-17.7500,-149.2833,72,
48,missoula,US,47.0166,-113.8009,35,
61,bethel,US,41.3712,-73.4140,44,
75,jamestown,US,42.0970,-79.2353,49,
78,alice springs,AU,-23.7000,133.8833,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
latitude = 0
longitude = 0
radius = 10000
categories = "accommodation.hotel"
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
limit = 1

# Set the parameters for the type of search

params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
         hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         # If no hotel is found, set the hotel name as "No hotel found".
         hotel_df.loc[index, "Hotel Name"] = "No hotel found"

     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
afaahiti - nearest hotel: Omati Lodge
missoula - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
jamestown - nearest hotel: DoubleTree Jamestown
alice springs - nearest hotel: Aurora Alice Springs
albany - nearest hotel: No hotel found
nkove - nearest hotel: No hotel found
bojnurd - nearest hotel: مرکز آموزشی رفاهی فرهنگیان بجنورد
dondo - nearest hotel: No hotel found
hwacheon - nearest hotel: 아쿠아틱리조트
karratha - nearest hotel: Karratha International Hotel
zhosaly - nearest hotel: Gostinitsa Yeseta
lucapa - nearest hotel: No hotel found
saynshand - nearest hotel: No hotel found
lompoc - nearest hotel: Red Roof Inn Lompoc
roma - nearest hotel: Wingate by Wyndham Rome
constantia - nearest hotel: Hotel Dali
yeonggwang - nearest hotel: 초원호텔
hwange - nearest hotel: No hotel found
metkovic - nearest hotel: Hotel MB
santa marta de ortigueira - nearest hotel: Hotel la Perla
morden - nearest hotel: Super 8 Hotel
shache - nearest hotel: 莎车新盛宾馆
chern

,City,Country,Lat,Lng,Humidity,Hotel Name
21,afaahiti,PF,-17.7500,-149.2833,72,Omati Lodge
48,missoula,US,47.0166,-113.8009,35,No hotel found
61,bethel,US,41.3712,-73.4140,44,Hampton Inn Danbury
75,jamestown,US,42.0970,-79.2353,49,DoubleTree Jamestown
78,alice springs,AU,-23.7000,133.8833,33,Aurora Alice Springs


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)